In [ ]:
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors

In [ ]:
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)
base.to("cuda")
#pipe.enable_xformers_memory_efficient_attention()
#pipe.enable_attention_slicing()
#pipe.enable_model_cpu_offload()

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

prompt = "a photo of beautiful cat,color white,blue eyes,Vincent van Gogh"

image = base(
    prompt=prompt,
    num_inference_steps=20,
    denoising_end=0.8,
    output_type="latent",
).images

refiner(
    prompt=prompt,
    num_inference_steps=20,
    denoising_start=0.8,
    image=image,
).images[0]